In [2]:
import sys
import DnnLib
import numpy as np

In [2]:
#Example 1

# Create sample data (XOR problem)
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float64)
y = np.array([[0], [1], [1], [0]], dtype=np.float64)


# Create a neural network: 2 -> 4 -> 1
layer1 = DnnLib.DenseLayer(2, 4, DnnLib.ActivationType.RELU)
layer2 = DnnLib.DenseLayer(4, 1, DnnLib.ActivationType.SIGMOID)

# Create optimizer
optimizer = DnnLib.Adam(learning_rate=0.01)

for epoch in range(100):
    # Forward pass
    h1 = layer1.forward(X)
    output = layer2.forward(h1)
    
    # Compute loss
    loss = DnnLib.mse(output, y)
    
    # Backward pass
    loss_grad = DnnLib.mse_gradient(output, y)
    grad2 = layer2.backward(loss_grad)
    grad1 = layer1.backward(grad2)
    
    # Update parameters
    optimizer.update(layer2)
    optimizer.update(layer1)
    
    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.6f}")

Epoch 0, Loss: 0.213275
Epoch 20, Loss: 0.187392
Epoch 40, Loss: 0.172109
Epoch 60, Loss: 0.168018
Epoch 80, Loss: 0.167389


In [3]:
#Dense Layers

# Basic dense layer: input_dim=3, output_dim=5, default ReLU activation
layer = DnnLib.DenseLayer(3, 5)

# With specific activation function
layer = DnnLib.DenseLayer(3, 5, DnnLib.ActivationType.SIGMOID)

#operations
single_input = np.array([1.0, 2.0, 3.0], dtype=np.float64)
batch_input = np.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]], dtype=np.float64)

output = layer.forward(single_input) # Shape: (5,)
batch_output = layer.forward(batch_input) # Shape: (2, 5)

# Forward pass without activation (linear only)
linear_output = layer.forward_linear(single_input)

# Backward pass (for gradient computation)
gradient_input = np.ones(5, dtype=np.float64) # Gradient from next layer
input_gradient = layer.backward(gradient_input)

# Access parameters
weights = layer.weights # Shape: (output_dim, input_dim)
bias = layer.bias # Shape: (output_dim,)

# Access gradients (after backward pass)
weight_grads = layer.weight_gradients
bias_grads = layer.bias_gradients

# Modify activation function
layer.activation_type = DnnLib.ActivationType.TANH

In [4]:
#binary classification
np.random.seed(42)
x = np.random.randn(1000, 2).astype(np.float64)
y = (x[:,0] + x[:, 1] > 0).astype(np.float64).reshape(-1,1)

print(f"Dataset: {x.shape[0]} samples, {x.shape[1]} features")
print(f"Positive class: {np.sum(y)}/{len(y)} samples")

layers = [
 DnnLib.DenseLayer(2, 8, DnnLib.ActivationType.RELU),
 DnnLib.DenseLayer(8, 4, DnnLib.ActivationType.RELU),
 DnnLib.DenseLayer(4, 1, DnnLib.ActivationType.SIGMOID)
]

# Use Adam optimizer
optimizer = DnnLib.Adam(learning_rate=0.01)

# Training loop
print("Training binary classifier...")
for epoch in range(200):
    # Forward pass
    h1 = layers[0].forward(x)
    h2 = layers[1].forward(h1)
    output = layers[2].forward(h2)
    
    # Binary cross-entropy loss
    loss = DnnLib.binary_cross_entropy(output, y)
    
    # Backward pass
    grad = DnnLib.binary_cross_entropy_gradient(output, y)
    grad = layers[2].backward(grad)
    grad = layers[1].backward(grad)
    grad = layers[0].backward(grad)
    
    # Update all layers
    optimizer.update(layers[2])
    optimizer.update(layers[1])
    optimizer.update(layers[0])
    
    if epoch % 40 == 0:
        # Calculate accuracy
        predictions = (output > 0.5).astype(np.float64)
        accuracy = np.mean(predictions == y)
        print(f"Epoch {epoch}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")
        
print("Training completed!")

Dataset: 1000 samples, 2 features
Positive class: 514.0/1000 samples
Training binary classifier...
Epoch 0, Loss: 0.9124, Accuracy: 0.4930
Epoch 40, Loss: 0.1307, Accuracy: 0.9870
Epoch 80, Loss: 0.0372, Accuracy: 0.9980
Epoch 120, Loss: 0.0269, Accuracy: 0.9970
Epoch 160, Loss: 0.0225, Accuracy: 0.9980
Training completed!


In [5]:
#Example 2
# Generate 3-class classification data
np.random.seed(42)
n_samples = 1500
n_features = 4
n_classes = 3

X = np.random.randn(n_samples, n_features).astype(np.float64)

# Create 3 classes with different patterns
y_labels = np.zeros(n_samples, dtype=int)
y_labels[:500] = 0 # Class 0
y_labels[500:1000] = 1 # Class 1
y_labels[1000:] = 2 # Class 2

# Convert to one-hot encoding
y = np.zeros((n_samples, n_classes), dtype=np.float64)
y[np.arange(n_samples), y_labels] = 1.0

print(f"Multi-class dataset: {X.shape}, {n_classes} classes")

# Create network: 4 -> 16 -> 8 -> 3
layers = [
DnnLib.DenseLayer(4, 16, DnnLib.ActivationType.RELU),
DnnLib.DenseLayer(16, 8, DnnLib.ActivationType.RELU),
DnnLib.DenseLayer(8, 3, DnnLib.ActivationType.SOFTMAX)
]

# Use RMSprop optimizer
optimizer = DnnLib.RMSprop(learning_rate=0.01, decay_rate=0.9)
print("Training multi-class classifier...")

for epoch in range(150):
# Forward pass
    h1 = layers[0].forward(X)
    h2 = layers[1].forward(h1)
    output = layers[2].forward(h2)
    
    # Cross-entropy loss
    loss = DnnLib.cross_entropy(output, y)
    
    # Backward pass
    grad = DnnLib.cross_entropy_gradient(output, y)
    grad = layers[2].backward(grad)
    grad = layers[1].backward(grad)
    grad = layers[0].backward(grad)
    
    # Update parameters
    for layer in layers:
        optimizer.update(layer)
        
    if epoch % 30 == 0:
        # Calculate accuracy
        predicted_classes = np.argmax(output, axis=1)
        accuracy = np.mean(predicted_classes == y_labels)
        print(f"Epoch {epoch}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")
        
print("Multi-class training completed!")


Multi-class dataset: (1500, 4), 3 classes
Training multi-class classifier...
Epoch 0, Loss: 8.9168, Accuracy: 0.3307
Epoch 30, Loss: 5.5706, Accuracy: 0.3453
Epoch 60, Loss: 7.2441, Accuracy: 0.3460
Epoch 90, Loss: 10.5551, Accuracy: 0.3453
Epoch 120, Loss: 14.0342, Accuracy: 0.3333
Multi-class training completed!


In [6]:
#example 3 - regression
# Generate regression data: y = x₁² + x₂² + noise
np.random.seed(42)
n_samples = 2000
X = np.random.uniform(-2, 2, size=(n_samples, 2)).astype(np.float64)
y = (X[:, 0]**2 + X[:, 1]**2 + 0.1 * np.random.randn(n_samples)).reshape(-1, 1)

print(f"Regression dataset: {X.shape} -> {y.shape}")
print(f"Target range: [{np.min(y):.2f}, {np.max(y):.2f}]")

# Create deeper network for regression: 2 -> 32 -> 16 -> 8 -> 1
layers = [
    DnnLib.DenseLayer(2, 32, DnnLib.ActivationType.RELU),
    DnnLib.DenseLayer(32, 16, DnnLib.ActivationType.RELU),
    DnnLib.DenseLayer(16, 8, DnnLib.ActivationType.RELU),
    DnnLib.DenseLayer(8, 1, DnnLib.ActivationType.RELU) 
    # No activation for regression output
]

# Try different optimizers
optimizers = [
("SGD", DnnLib.SGD(0.001)),
("SGD+Momentum", DnnLib.SGD(0.001, 0.9)),
("Adam", DnnLib.Adam(0.001)),
("RMSprop", DnnLib.RMSprop(0.001))
]

for opt_name, optimizer in optimizers:
    print(f"\n--- Training with {opt_name} ---")
    # Reset network weights (create new layers)
    layers = [
        DnnLib.DenseLayer(2, 32, DnnLib.ActivationType.RELU),
        DnnLib.DenseLayer(32, 16, DnnLib.ActivationType.RELU),
        DnnLib.DenseLayer(16, 8, DnnLib.ActivationType.RELU),
        DnnLib.DenseLayer(8, 1, DnnLib.ActivationType.RELU)
    ]
    optimizer.reset()
    
    # Training
    for epoch in range(100):
        # Forward pass
        activation = X
        
        for layer in layers:
            activation = layer.forward(activation)
        output = activation
        
        # MSE loss for regression
        loss = DnnLib.mse(output, y)
        
        # Backward pass
        grad = DnnLib.mse_gradient(output, y)
        for layer in reversed(layers):
            grad = layer.backward(grad)
            optimizer.update(layer)
        if epoch % 25 == 0:
        # Calculate R² score
            ss_res = np.sum((y - output) ** 2)
            ss_tot = np.sum((y - np.mean(y)) ** 2)
            r2_score = 1 - (ss_res / ss_tot)
            print(f" Epoch {epoch}, Loss: {loss:.6f}, R²: {r2_score:.4f}")
    print(f" Final {opt_name} Loss: {loss:.6f}")

Regression dataset: (2000, 2) -> (2000, 1)
Target range: [-0.19, 7.95]

--- Training with SGD ---
 Epoch 0, Loss: 10.165082, R²: -2.5072
 Epoch 25, Loss: 10.165082, R²: -2.5072
 Epoch 50, Loss: 10.165082, R²: -2.5072
 Epoch 75, Loss: 10.165082, R²: -2.5072
 Final SGD Loss: 10.165082

--- Training with SGD+Momentum ---
 Epoch 0, Loss: 10.011617, R²: -2.4542
 Epoch 25, Loss: 7.035337, R²: -1.4274
 Epoch 50, Loss: 0.571781, R²: 0.8027
 Epoch 75, Loss: 0.249464, R²: 0.9139
 Final SGD+Momentum Loss: 0.175653

--- Training with Adam ---
 Epoch 0, Loss: 165.291529, R²: -56.0296
 Epoch 25, Loss: 25.489426, R²: -7.7945
 Epoch 50, Loss: 7.915615, R²: -1.7311
 Epoch 75, Loss: 6.809272, R²: -1.3494
 Final Adam Loss: 6.448034

--- Training with RMSprop ---
 Epoch 0, Loss: 419.796687, R²: -143.8400
 Epoch 25, Loss: 74.152471, R²: -24.5844
 Epoch 50, Loss: 19.237093, R²: -5.6373
 Epoch 75, Loss: 8.708759, R²: -2.0047
 Final RMSprop Loss: 6.031800
